In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [2]:
data = pd.read_csv("..//..//dataset//oakland-crime-statistics-2011-to-2016//records-for-2015.csv")
print(data)

       Agency              Create Time                      Location Area Id  \
0          OP  2015-01-01T00:01:59.000              S ELMHURST AV         P3   
1          OP  2015-01-01T00:02:02.000                   AV&D ST          P3   
2          OP  2015-01-01T00:02:06.000               BANCROFT AV          P3   
3          OP  2015-01-01T00:03:16.000            MACARTHUR BLVD          P3   
4          OP  2015-01-01T00:03:45.000             ST&ADELINE ST          P1   
5          OP  2015-01-01T00:03:54.000            MACARTHUR BLVD          P3   
6          OP  2015-01-01T00:04:05.000                   36TH AV          P3   
7          OP  2015-01-01T00:04:14.000            17TH ST&41ST AV         P3   
8          OP  2015-01-01T00:04:15.000         AV&MACARTHUR BLVD          P3   
9          OP  2015-01-01T00:04:38.000                CASWELL AV          P3   
10         OP  2015-01-01T00:04:50.000                   83RD AV          P3   
11         OP  2015-01-01T00:04:58.000  

#### 1. 删除空值处理

In [3]:
#删除所有属性中有空值的行
data1 = data.dropna()
print(data1)

       Agency              Create Time                      Location Area Id  \
0          OP  2015-01-01T00:01:59.000              S ELMHURST AV         P3   
1          OP  2015-01-01T00:02:02.000                   AV&D ST          P3   
2          OP  2015-01-01T00:02:06.000               BANCROFT AV          P3   
3          OP  2015-01-01T00:03:16.000            MACARTHUR BLVD          P3   
4          OP  2015-01-01T00:03:45.000             ST&ADELINE ST          P1   
5          OP  2015-01-01T00:03:54.000            MACARTHUR BLVD          P3   
6          OP  2015-01-01T00:04:05.000                   36TH AV          P3   
7          OP  2015-01-01T00:04:14.000            17TH ST&41ST AV         P3   
8          OP  2015-01-01T00:04:15.000         AV&MACARTHUR BLVD          P3   
9          OP  2015-01-01T00:04:38.000                CASWELL AV          P3   
10         OP  2015-01-01T00:04:50.000                   83RD AV          P3   
11         OP  2015-01-01T00:04:58.000  

#### 2. 用最高频率值进行填充

In [4]:
data2 = copy.deepcopy(data)#拷贝原数据集用于填充
for i in range(10):
    count = data2.iloc[:,i].value_counts()  #依次获取每一列的series
    name = pd.DataFrame(data2.iloc[:,i]).columns.values.tolist()
    sort_ans = count.sort_values(ascending = False) #对各类别的频数值进行排序 并选择最高频率的索引
    fre = sort_ans.index[0]
    fre = str(fre)
    print(name[0]+"的最高频率值为"+fre+'\n')
    
    data2[name[0]] = data2[name[0]].fillna(fre) #进行缺失值的填充
print("------------------------------------------"+'\n'+"填充完毕")

Agency的最高频率值为OP

Create Time的最高频率值为2015-04-18T13:52:06.000

Location的最高频率值为 INTERNATIONAL BLVD    

Area Id的最高频率值为P3

Beat的最高频率值为04X

Priority的最高频率值为2

Incident Type Id的最高频率值为933R

Incident Type Description的最高频率值为ALARM-RINGER

Event Number的最高频率值为LOP150426000348

Closed Time的最高频率值为2015-08-16T04:18:05.000

------------------------------------------
填充完毕


In [5]:
print(data2)  #填充结果

       Agency              Create Time                      Location Area Id  \
0          OP  2015-01-01T00:01:59.000              S ELMHURST AV         P3   
1          OP  2015-01-01T00:02:02.000                   AV&D ST          P3   
2          OP  2015-01-01T00:02:06.000               BANCROFT AV          P3   
3          OP  2015-01-01T00:03:16.000            MACARTHUR BLVD          P3   
4          OP  2015-01-01T00:03:45.000             ST&ADELINE ST          P1   
5          OP  2015-01-01T00:03:54.000            MACARTHUR BLVD          P3   
6          OP  2015-01-01T00:04:05.000                   36TH AV          P3   
7          OP  2015-01-01T00:04:14.000            17TH ST&41ST AV         P3   
8          OP  2015-01-01T00:04:15.000         AV&MACARTHUR BLVD          P3   
9          OP  2015-01-01T00:04:38.000                CASWELL AV          P3   
10         OP  2015-01-01T00:04:50.000                   83RD AV          P3   
11         OP  2015-01-01T00:04:58.000  

#### 3. 根据数据对象的相似性来填充数据值

   这里我们使用的是KNN算法，实体之间的距离计算使用曼哈顿距离。K的值设置为6。我们选取与存在空缺值的向量距离最近的6个向量在对应属性上的平均值来对空缺值进行填充。